<a href="https://colab.research.google.com/github/CaioPassos3/TrabalhoIA/blob/main/TrabIntArt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Carregue o arquivo CSV
file_path = '/content/sample_data/Housing.csv'
data = pd.read_csv(file_path)

In [ ]:
# Verificar as primeiras linhas do dataset
print(data.head())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [ ]:
# Identificar as variáveis categóricas
categorical_features = data.select_dtypes(include=['object']).columns
print(categorical_features)


Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')


In [ ]:
# Identificar as variáveis categóricas
categorical_features = data.select_dtypes(include=['object']).columns
print(categorical_features)

Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')


In [ ]:
# Remover as variáveis categóricas originais
data = data.drop(categorical_features, axis=1)

In [ ]:
# Aplicar One-Hot Encoding
data = pd.get_dummies(data, columns=data.select_dtypes(include=['object']).columns, drop_first=True)
print(data.head())

      price  area  bedrooms  bathrooms  stories  parking
0  13300000  7420         4          2        3        2
1  12250000  8960         4          4        4        3
2  12250000  9960         3          2        2        2
3  12215000  7500         4          2        2        3
4  11410000  7420         4          1        2        2


In [ ]:
# Separar variáveis preditivas e alvo
X = data.drop('price', axis=1)
y = data['price']


In [ ]:
# Separar em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print(f'Tamanho do conjunto de treino: {X_train.shape}')
print(f'Tamanho do conjunto de teste: {X_test.shape}')

Tamanho do conjunto de treino: (436, 5)
Tamanho do conjunto de teste: (109, 5)


In [ ]:
# Implemente a regressão linear ao conjunto de dados usando o método dos mínimos quadrados (OLS)
def ols(X, y):
    X_T = X.T
    X_TX = np.dot(X_T, X)
    X_Ty = np.dot(X_T, y)
    beta = np.linalg.inv(X_TX).dot(X_Ty)
    return beta

    beta_ols = ols(X_train, y_train)
y_pred_ols = np.dot(X_test, beta_ols)

In [ ]:
# Avalie o desempenho do modelo usando as métricas MSE e R²
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def r2(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred) * 2) / np.sum((y_true - np.mean(y_true)) * 2))

mse_ols = mse(y_test, y_pred_ols)
r2_ols = r2(y_test, y_pred_ols)
print(f'MSE (OLS): {mse_ols:.2f}')
print(f'R² (OLS): {r2_ols:.2f}')

MSE (OLS): 2288011803457.03
R² (OLS): -1646553360794443.75


In [ ]:
# Implemente a regressão linear ao conjunto de dados usando o método do gradiente descendente (GD)
def gradient_descent(X, y, learning_rate, n_iters):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0
    for _ in range(n_iters):
        y_pred = np.dot(X, weights) + bias
        dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
        db = (1 / n_samples) * np.sum(y_pred - y)
        weights -= learning_rate * dw
        bias -= learning_rate * db
    return weights, bias

learning_rate = 0.01
n_iters = 1000
weights_gd, bias_gd = gradient_descent(X_train, y_train, learning_rate, n_iters)
y_pred_gd = np.dot(X_test, weights_gd) + bias_gd

<ipython-input-18-60bc2b8727cc>:10: RuntimeWarning: invalid value encountered in subtract
  weights -= learning_rate * dw


In [ ]:
# Avalie o desempenho do modelo usando as métricas MSE e R²
mse_gd = mse(y_test, y_pred_gd)
r2_gd = r2(y_test, y_pred_gd)
print(f'MSE (GD): {mse_gd:.2f}')
print(f'R² (GD): {r2_gd:.2f}')

MSE (GD): nan
R² (GD): 1.00


In [ ]:
# Normalize as variáveis preditivas usando o método min-max e a variável alvo usando o método z-score
def min_max_scaler(X):
    return (X - X.min()) / (X.max() - X.min())

def z_score_scaler(y):
    return (y - y.mean()) / y.std()

X_train_scaled = min_max_scaler(X_train)
y_train_scaled = z_score_scaler(y_train)
X_test_scaled = min_max_scaler(X_test)


In [ ]:
# Repita o procedimento do item A com os dados normalizados
weights_gd_scaled, bias_gd_scaled = gradient_descent(X_train_scaled, y_train_scaled, learning_rate, n_iters)
y_pred_gd_scaled = np.dot(X_test_scaled, weights_gd_scaled) + bias_gd_scaled

In [ ]:
# Avalie o desempenho do modelo usando as métricas MSE e R²
mse_gd_scaled = mse(y_test, y_pred_gd_scaled)
r2_gd_scaled = r2(y_test, y_pred_gd_scaled)
print(f'M')

M
